In [ ]:
# 합성곱 신경망convolutional neural netwrok, CNN
'''
지금까지 배웠던 신경망과 유사하지만, 합성곱 계층convolutional layer과 풀링 계층pooling layer이 새로 등장한다.
이미지 인식과 음성 인식 등에 사용되는 신경망. 특히 이미지 인식 분야는 대부분 CNN을 기초로 한다.
* 합성곱은 공학과 물리학에서 널리 쓰이는 수학적 개념으로, '두 함수 중 하나를 반전(reverse), 이동(shift)시켜가며 나머지 함수와의 곱을 연이어 적분한다'는 개념이다.
'''

'''
지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었으며, 이를 완전연결fully-connected, 전결합이라고 한다. 또한, 완전히 연결된 계층을 Affine 계층이라는 이름으로 구현했다.
따라서 'Affine-활성화함수'가 한 층을 이루며, 출력 계층에서 'Affine-Softmax'조합을 가지고 있었다.
이 때, 인접하는 계층의 뉴런이 모두 연결되고, 출력의 수는 임의로 정할 수 있다.
'''
'''
그러나 CNN에서는 합성곱 계층과 풀링계층이 추가되어, 'Conv(합성곱 계층)-활성화함수-(Pooling)'이 한 층을 이룬다. 이 때, Pooling은 생략하기도 한다.
또한, 출력 층에 가까운 층은 'Affine-활성화함수'층을 그대로 사용하기도 하며, 마지막 출력 계층에서는 'Affine-Softmax'조합을 그대로 사용한다.
'''

In [ ]:
# 완전연결 계층(Affine 계층)의 문제점
'''
'데이터의 형상'이 무시된다.
예를 들어 입력 데이터가 이미지인 경우, 이미지는 통상 가로*세로*채널(색상)으로 구성된 3차원 데이터이나, 완전 연결 계층에 입력된 경우 3차원 데이터를 1차원 데이터로 평탄화(flatten)해줘야 한다.
이미지는 3차원 형상이며, 이 형상 자체에는 공간적 정보가 담겨있다. 예를 들어 공간적으로 가까운 픽셀 값은 비슷하거나, RGB의 각 채널이 밀접하게 연관되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없을 수 있다.
그러나 완전연결 계층은 이를 모두 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 보므로, 형상에 담긴 공간적 정보를 살릴 수 없다.
'''
# 합성곱 계층은 형상을 유지한다.
# 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다. 따라서 형상을 가진 데이터를 제대로 이해할 (가능성이 있는) 것이다.

In [ ]:
# CNN에서는 합성곱 계층의 입출력 데이터를 특징 맵feature map이라고 한다. 입력 특징 맵, 출력 특징 맵 등으로 부른다.

# 합성곱 계층에서는 합성곱 연산을 처리하며, 이는 이미지 처리에서 말하는 필터(커널) 연산에 해당한다.
'''
필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용한다.
이 때 적용이라 함은, 대응하는 원소끼리 서로 곱한 후 그 총합을 구한다는 의미이다(이 계산을 단일 곱셈-누산fused multiply-add, FMA이라고 한다).
그리고 그 결과를 출력의 해당 장소에 저장한다. 이 과정을 모든 장소에서 수행하면 합성 곱 연산의 출력이 완성된다.
'''
# 완전연결 신경망에서 가중치 매개변수와 편향이 존재하듯, CNN에서는 필터의 매개변수가 그 동안의 '가중치'에 해당한다.
# 또한, CNN에도 편향이 존재해서, 하나(1*1)의 편향 값이 필터를 적용한 후에 각 데이터에 더해진다.

In [2]:
# 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예컨대 0)으로 채우기도 하는데, 이를 패딩padding이라고 한다. 자주 이용하는 기법이다.
# 패딩을 통해 출력 크기(x*y)를 조정할 수 있다. 정확히 말하면, 패딩을 크게 하면 출력 크기가 커진다.
# 필터를 적용할수록 출력이 작아지므로, 합성곱 연산을 몇번이나 되풀이하는 심층 신경망에서 출력 크기가 너무 줄어들지 않도록 하기 위해 사용한다.

# 필터를 적용하는 위치 간격을 스트라이드stride(보폭)라고 하며, 스트라이드를 적용하면 필터 윈도우의 이동 간격이 스트라이드로 늘어난다.
# 스트라이드를 키우면 출력의 크기가 줄어든다.

In [ ]:
# 3차원 데이터의 합성곱 연산
# 2차원 데이터와 비교하면, 길이 방향(채널 방향)으로 특징 맵 수가 늘어난다.
# 따라서 '특징 맵 수에 맞게 필터 수를 조정'하고, 그 결과를 더해서 하나의 출력을 얻는다.

'''
데이터와 필터를 직육면체 블록으로 생각한다면 쉽다.
3차원 데이터를 (채널 C, 높이 H, 너비 W)로 생각하고, 필터도 같은 순서로 쓴다. 예를 들어 채널 수 C, 필터 높이 FH, 필터 너비 FW의 경우 (C, FH, FW)로 쓴다.
(C, H, W) * (C, FH, FW) = (1, OH, OW)로 출력데이터가 나온다. 즉, 출력 데이터는 채널 수가 1인 특징 맵이다.

그러나 심층 신경망에서는 출력 데이터를 다수의 채널로 내보내야 다음 층으로 넘길 수 있으므로, 필터를 FN개 적용하여 출력 데이터를 (FN, OH, OW)로 조정해준다.
따라서 필터의 가중치는 FN, C, FH, FW를 모두 고려해야 하므로 4차원 가중치 데이터가 된다(FN, C, FH, FW).

필터를 적용 후 (FN, OH, OW)가 되므로, 이 값에 편향을 더하여 출력 데이터 (FN, OH, OW)를 만들어주어야 한다. 따라서 편향값은 모든 채널에 더할 수 있도록 (FN, 1, 1)형상이다.

이처럼 형상이 다른 블록의 덧셈은 numpy의 브로드캐스트 기능으로 쉽게 구현할 수 있다.
'''
# 전체적인 3차원 데이터의 합성곱 연산 과정은 다음과 같다.
# 입력 데이터 * 필터 + 편향 = 출력 데이터
# ((C, H, W) * (FN, C, FH, FW)) + (FN, 1, 1) = (FN, OH, OW)

In [3]:
# 신경망 처리에서 입력 데이터를 묶어 하나의 배치로 처리하듯, 합성곱 연산도 배치 처리를 지원해보자.
# 각 계층을 흐르는 데이터 차원을 하나 늘려, 데이터 수를 매개변수로 추가한다. (데이터 수 N, C, H, W)형태.
# ((N, C, H, W) * (FN, C, FH, FW)) + (FN, 1, 1) = (N, FN, OH, OW)
# 이를 통해 N개의 데이터 블록에 대한 처리, 즉 N회분의 데이터 처리를 한번에 수행할 수 있다.

In [ ]:
# 풀링은 가로, 세로 방향의 공간을 줄이는 연산으로, 여러 개의 칸에서 값을 뽑아내 하나의 칸으로 축소시킨다.
# n*n 최대풀링max pooling은 일반적으로 스트라이드 n으로 처리하며, n*n개의 칸에서 가장 큰 값 하나를 뽑아내 하나의 칸으로 축소시킨다.
# 최대 풀링 이외에도 평균 풀링등이 있다. 이미지 인식 분야에서는 주로 최대 풀링을 사용한다.

# 풀링 연산의 특징
'''
학습해야 할 매개 변수가 없다 - 대상영역에서 최댓값이나 평균을 취하는 명확한 처리이므로 따로 학습할 필요가 없다.
채널 수가 변하지 않는다 - 각 채널마다 독립적으로 계산하므로 연산 후 채널 수 자체는 바뀌지 않는다.
입력의 변화에 영향을 적게 받는다 - 입력 데이터가 약간 어긋나더라도(예를 들어 오른쪽으로 1칸씩 이동) 이 차이를 풀링이 흡수해 사라지게 할 수도 있다.
'''